In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
top_institutions = {
    "0001067983": "Berkshire Hathaway",
    #"0001166559": "Renaissance Technologies"
    #"0001037389": "Citadel Advisors",
    #"0001081060": "BlackRock Inc.",
    #"0001103804": "Bridgewater Associates",
    #"0000922971": "Two Sigma Investments",
    #"0001079114": "Millennium Management",
    #"0000912057": "Vanguard Group",
    #"0000316927": "FMR LLC",
    #"0000354204": "State Street Corp"
}

In [7]:
def fetch_filing_metadata(cik, institution_name):
    url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type=13F-HR&owner=exclude&count=40&output=atom"
    headers = {
        "User-Agent": "DataScienceInternshipBot/1.0 (contact: chandanarchutha.n@gmail.com)",
        "Accept": "application/xml",
        "Accept-Language": "en-US,en;q=0.5",
        "Referer": "https://www.sec.gov/"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "xml")
    entries = soup.find_all("entry")

    filing_links = []
    filed_dates = []
    institution_names = []

    for entry in entries:
        link = entry.find("link")["href"].strip()
        date = entry.find("updated").text.strip()
        filing_links.append(link)
        filed_dates.append(date)
        institution_names.append(institution_name)

    return filing_links, filed_dates, institution_names

In [9]:
# Function to extract XML info table link from index page
def extract_info_table_xml(index_url):
    headers = {
        "User-Agent": "DataScienceInternshipBot/1.0 (contact: chandanarchutha.n@gmail.com)",
        "Accept-Language": "en-US,en;q=0.9"
    }

    try:
        response = requests.get(index_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        table = soup.find("table", class_="tableFile")
        if table:
            for row in table.find_all("tr"):
                cols = row.find_all("td")
                if len(cols) >= 3:
                    a_tag = cols[2].find("a")
                    doc_type = cols[3].text.strip().lower()
                    if a_tag:
                        href = a_tag.get("href", "")
                        if href.endswith(".xml") and "form13f" in href.lower() and doc_type == "information table":
                            return "https://www.sec.gov" + href
    except Exception as e:
        print(f"❌ Error parsing {index_url}: {e}")
    return None

In [11]:
def extract_13f_holdings_from_html(xml_url, filed_date, institution_name):
    headers = {
        "User-Agent": "DataScienceInternshipBot/1.0 (contact: chandanarchutha.n@gmail.com)",
        "Accept": "application/xml",
        "Accept-Language": "en-US,en;q=0.9"
    }

    response = requests.get(xml_url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    tables = soup.find_all("table")
    if not tables:
        return []

    data_table = tables[-1]
    rows = data_table.find_all("tr")
    holdings = []

    for row in rows[4:]:
        cols = row.find_all("td")
        if len(cols) >= 13:
            issuer = cols[0].text.strip()
            class_title = cols[1].text.strip()
            cusip = cols[2].text.strip()
            value = cols[4].text.strip()
            shares = cols[5].text.strip()
            discretion = cols[8].text.strip()
            voting_sole = cols[10].text.strip()
            voting_shared = cols[11].text.strip()
            voting_none = cols[12].text.strip()

            holdings.append([
                issuer, class_title, cusip, value, shares, discretion,
                voting_sole, voting_shared, voting_none, filed_date, xml_url, institution_name
            ])
    return holdings


all_holdings = []

for cik, institution_name in top_institutions.items():
    filing_links, filed_dates, institution_names = fetch_filing_metadata(cik, institution_name)
    xml_links = [extract_info_table_xml(link) for link in filing_links]
    sample_filings = list(zip(xml_links, filed_dates, institution_names))

    for xml_url, filed_date, institution_name in sample_filings:
        if xml_url:
            try:
                extracted = extract_13f_holdings_from_html(xml_url, filed_date, institution_name)
                all_holdings.extend(extracted)
            except Exception as e:
                print(f"Error parsing {xml_url}: {e}")

In [13]:
import time

# Final version of scraping loop with improvements
all_holdings = []
failed_urls = []

for cik, institution_name in top_institutions.items():
    filing_links, filed_dates, institution_names = fetch_filing_metadata(cik, institution_name)
    xml_links = [extract_info_table_xml(link) for link in filing_links]
    sample_filings = list(zip(xml_links, filed_dates, institution_names))
    for i, (xml_url, filed_date, institution_name) in enumerate(sample_filings):
        if xml_url:
            try:
                extracted = extract_13f_holdings_from_html(xml_url, filed_date, institution_name)
                all_holdings.extend(extracted)
                if i % 5 == 0:
                    print(f"Processed {i+1}/{len(sample_filings)} filings for {institution_name}...")
                time.sleep(0.5)  # polite pause between requests to avoid SEC throttling
            except Exception as e:
                print(f"Error parsing {xml_url}: {e}")
                failed_urls.append((xml_url, institution_name))



    
if failed_urls:
    print(f"\n❌ {len(failed_urls)} filings failed to parse. You can retry them later if needed.")
    for url, name in failed_urls:
        print(f"- {name}: {url}")



Processed 1/40 filings for Berkshire Hathaway...
Processed 6/40 filings for Berkshire Hathaway...
Processed 11/40 filings for Berkshire Hathaway...
Processed 16/40 filings for Berkshire Hathaway...
Processed 21/40 filings for Berkshire Hathaway...
Processed 26/40 filings for Berkshire Hathaway...
Processed 31/40 filings for Berkshire Hathaway...
Processed 36/40 filings for Berkshire Hathaway...


In [15]:
df = pd.DataFrame(all_holdings, columns=[
    "Issuer", "Class", "CUSIP", "Value (x$1000)", "Shares", "Discretion",
    "Voting - Sole", "Voting - Shared", "Voting - None", "Filed Date", "Source URL", "Institution"
])        

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1530 entries, 0 to 1529
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Issuer           1530 non-null   object
 1   Class            1530 non-null   object
 2   CUSIP            1530 non-null   object
 3   Value (x$1000)   1530 non-null   object
 4   Shares           1530 non-null   object
 5   Discretion       1530 non-null   object
 6   Voting - Sole    1530 non-null   object
 7   Voting - Shared  1530 non-null   object
 8   Voting - None    1530 non-null   object
 9   Filed Date       1530 non-null   object
 10  Source URL       1530 non-null   object
 11  Institution      1530 non-null   object
dtypes: object(12)
memory usage: 143.6+ KB
